# Candlestick body & pattern & return relationship

In [15]:
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import os
from pathlib import Path
import pandas_ta as ta
import pspriceaction.price_action as pa
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [16]:
data = []
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
algo_dir = current_dir.parent.parent
transform_csv_file = str(current_dir) + '/VN30F1M_5minutes_pattern.csv'
if os.path.isfile(transform_csv_file):
    data = pd.read_csv(transform_csv_file, index_col='Date', parse_dates=True)
    data['model'] = data['model'].fillna(value='')
else:
    csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
    is_file = os.path.isfile(csv_file)
    if is_file:
        dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
    else:
        dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)
    data = dataset.copy()
    data = pa.pattern_modeling(data)
    data.to_csv(transform_csv_file)

In [17]:
### Ignore this year data => tranh over fitting
data = data[data.index < '2024-01-01 00:00:00']

### Calculating trend

In [18]:
data["ma_line"] = data["Close"].rolling(20).mean()
data['above_ma'] = data.apply(lambda r: 1 if r['Close'] > r['ma_line'] else 0, axis=1)
data['below_ma'] = data.apply(lambda r: 1 if r['Close'] < r['ma_line'] else 0, axis=1)
data['total_above_ma'] = data['above_ma'].rolling(150).sum()
data['total_below_ma'] = data['below_ma'].rolling(150).sum()
data['total_above_ma_1w'] = data['above_ma'].rolling(250).sum()
data['total_below_ma_1w'] = data['below_ma'].rolling(250).sum()
data['trend_3d'] = data.apply(lambda r: 'switch' if r['total_above_ma'] == r['total_below_ma'] else ('up' if r['total_above_ma'] > r['total_below_ma'] else 'down'), axis=1)
data['trend_1w'] = data.apply(lambda r: 'switch' if r['total_above_ma_1w'] == r['total_below_ma_1w'] else ('up' if r['total_above_ma_1w'] > r['total_below_ma_1w'] else 'down'), axis=1)

### Calculating return

In [29]:
from cal_return_4_patterns import cal_return

In [20]:
%%time
data = cal_return(data)

CPU times: user 17.7 s, sys: 715 ms, total: 18.4 s
Wall time: 18.4 s


### Return Analytics

In [21]:
has_return = data[data['return'] != '']
#----
long_return = has_return[has_return['signal'] == 'long']
short_return = has_return[has_return['signal'] == 'short']
#----
negative_return = has_return[has_return['return'] < 0]
positive_return = has_return[has_return['return'] > 0]
#----
short_negative_return = short_return[short_return['return'] < 0]
short_positive_return = short_return[short_return['return'] > 0]
long_negative_return = long_return[long_return['return'] < 0]
long_positive_return = long_return[long_return['return'] > 0]

## Pattern model & return relationship

### Group by model & trend

In [22]:
has_return_group = has_return[['return']].groupby([has_return.model, has_return.trend_1w])

### Sum return

In [23]:
sum_group = has_return_group.sum()
sum_group['return'].sort_values()

model                                trend_1w
bullish_separating_line              up           -61.6
                                     down         -60.5
falling_three                        switch        -8.4
bullish_neck                         up            -3.3
bearish_gap, falling_three           up            -3.1
rising_n                             switch          -3
bullish_gap, fair_value_rising_gap   switch          -3
bearish_gap                          switch          -3
bullish_gap                          switch         2.0
rising_three                         switch         2.6
bullish_gap, rising_n                up             3.1
bullish_gap, rising_three            up             4.5
rising_n                             down           5.0
bearish_gap, falling_n               down             6
falling_three                        up             7.2
bearish_gap, falling_three           down           8.0
falling_n                            switch        10.5
be

### Mean group

In [24]:
mean_group = has_return_group.mean()
mean_group['return'].sort_values()

model                                trend_1w
bearish_gap                          switch          -3.0
rising_n                             switch          -3.0
bullish_gap, fair_value_rising_gap   switch          -3.0
falling_three                        switch          -2.1
bearish_gap, falling_three           up             -1.55
bullish_separating_line              down       -0.315104
                                     up          -0.30495
bullish_neck                         up         -0.004641
falling_three                        up          0.058537
bearish_neck                         up           0.07936
rising_n                             down         0.09434
bearish_gap                          up          0.198837
bullish_neck                         down         0.22217
bullish_gap                          down        0.411828
bullish_gap, fair_value_rising_gap   down        0.442667
bearish_gap, fair_value_falling_gap  up          0.461972
fair_value_falling_gap    

### Count return

In [25]:
count_group = has_return_group.count()
count_group['return'].sort_values()

model                                trend_1w
bearish_gap                          switch         1
bullish_gap                          switch         1
rising_n                             switch         1
bullish_gap, fair_value_rising_gap   switch         1
falling_n                            switch         2
bullish_gap, rising_three            up             2
bearish_gap, falling_n               down           2
bearish_gap, falling_three           up             2
bullish_gap, rising_n                up             2
bearish_gap, falling_three           down           3
falling_three                        switch         4
rising_three                         switch         5
bullish_separating_line              switch         5
bearish_neck                         switch        17
bullish_neck                         switch        18
fair_value_falling_gap               switch        29
fair_value_rising_gap                switch        42
rising_n                            

### Merge mean & count

In [26]:
k = mean_group[['return']].rename(columns={'return': 'mean'})
f = count_group[['return']].rename(columns={'return': 'count'})
y = pd.merge(k, f, left_index=True, right_index=True, how="left")

In [27]:
y.sort_values(by='count')

,,mean,count
model,trend_1w,,
bearish_gap,switch,-3.0,1
bullish_gap,switch,2.0,1
rising_n,switch,-3.0,1
"bullish_gap, fair_value_rising_gap",switch,-3.0,1
falling_n,switch,5.25,2
"bullish_gap, rising_three",up,2.25,2
"bearish_gap, falling_n",down,3.0,2
"bearish_gap, falling_three",up,-1.55,2
"bullish_gap, rising_n",up,1.55,2


In [28]:
y = y[y['mean'] > 0.3]
y.sort_values(by='count')

,,mean,count
model,trend_1w,,
bullish_gap,switch,2.0,1
falling_n,switch,5.25,2
"bearish_gap, falling_n",down,3.0,2
"bullish_gap, rising_n",up,1.55,2
"bullish_gap, rising_three",up,2.25,2
"bearish_gap, falling_three",down,2.666667,3
bullish_separating_line,switch,4.94,5
rising_three,switch,0.52,5
bearish_neck,switch,0.623529,17


### Group signal & trend

In [30]:
has_return_group_2 = has_return[['return']].groupby([has_return.signal, has_return.trend_1w])

In [31]:
mean_group_x = has_return_group_2.mean()
mean_group_x['return'].sort_values()

signal  trend_1w
short   up          0.352904
long    down        0.417836
        up          0.589051
short   down        0.795995
        switch      1.069811
long    switch      1.809589
Name: return, dtype: object